This notebook serves to extract information from parsed PDF text. The steps are as follows.

1. Filter sentences with numbers, new lines in them (if aim is to extract number + key metrics)
2. Remove stop words, punctuations, year etc
3. Apply part of speech tagging - generate some rules that will allow for extraction of number and metrics. To remember that negative words means that a minus needs to be added in front of the number
4. Output for each source -> The metrics, and value

# Import Packages

In [54]:
import pandas as pd
from utils import *
import json

-------

# Rule mining

test.txt from compiled sentences from word embeddings

In [143]:
df = pd.read_csv('../data/mining/text_ext_labelled_final.csv')
df = df.astype(str)
df = df.fillna("")
test_set_preproc_no_lemma, test_set_preproc_lemma  = [], []
for row in df.iterrows():
    line, labelled = row[1][0], [i for i in row[1][1:]]
    line = line.replace("\n", "")
    # append no lemma version
    test_set_preproc_no_lemma.append([line.lower(), preprocess(line.lower(), False), labelled])
    #lemmatize
    #append lemma version
    test_set_preproc_lemma.append([line.lower(), preprocess(line.lower(), True), labelled])

In [144]:
tags_lemma = process_list(test_set_preproc_lemma)
tags_no_lemma = process_list(test_set_preproc_no_lemma)

In [81]:
a,b,c = pos_extraction('Impact Investment Fund Climate & Biodiversity size double to us$ 350 million')
pos_printer(a,b,c)

Impact -> PROPN,NNP
Investment -> PROPN,NNP
Fund -> PROPN,NNP
Climate -> PROPN,NNP
& -> CCONJ,CC
Biodiversity -> PROPN,NNP
size -> NOUN,NN
double -> ADJ,JJ
to -> ADP,IN
us$ -> SYM,$
350 -> NUM,CD
million -> NUM,CD


In [82]:
a,b,c = pos_extraction('by 2025 , AP7 will have double the proportion of green investment compare with 2020 .')
pos_printer(a,b,c)

by -> ADP,IN
2025 -> NUM,CD
, -> PUNCT,,
AP7 -> PROPN,NNP
will -> VERB,MD
have -> AUX,VB
double -> DET,PDT
the -> DET,DT
proportion -> NOUN,NN
of -> ADP,IN
green -> ADJ,JJ
investment -> NOUN,NN
compare -> VERB,VBP
with -> ADP,IN
2020 -> NUM,CD
. -> PUNCT,.


In [83]:
a,b,c = pos_extraction('consistent with BlackRocks goal to double offering of sustainable etf ( to 150 ) , iShares launch over 45 new sustainable etf across the US , Europe , and Canada in 2020 .')
pos_printer(a,b,c)

consistent -> ADJ,JJ
with -> ADP,IN
BlackRocks -> PROPN,NNP
goal -> NOUN,NN
to -> PART,TO
double -> ADJ,JJ
offering -> NOUN,NN
of -> ADP,IN
sustainable -> ADJ,JJ
etf -> NOUN,NN
( -> PUNCT,-LRB-
to -> ADP,IN
150 -> NUM,CD
) -> PUNCT,-RRB-
, -> PUNCT,,
iShares -> PROPN,NNP
launch -> VERB,VBP
over -> ADP,IN
45 -> NUM,CD
new -> ADJ,JJ
sustainable -> ADJ,JJ
etf -> NOUN,NN
across -> ADP,IN
the -> DET,DT
US -> PROPN,NNP
, -> PUNCT,,
Europe -> PROPN,NNP
, -> PUNCT,,
and -> CCONJ,CC
Canada -> PROPN,NNP
in -> ADP,IN
2020 -> NUM,CD
. -> PUNCT,.


In [84]:
a,b,c = pos_extraction('-PRON- client from around the world recently report plan to double -PRON- sustainable asset over the next five year with climate change highlight as the most prominent sustainability issue .')
pos_printer(a,b,c)

-PRON- -> PUNCT,:
client -> NOUN,NN
from -> ADP,IN
around -> ADP,IN
the -> DET,DT
world -> NOUN,NN
recently -> ADV,RB
report -> VERB,VBP
plan -> NOUN,NN
to -> PART,TO
double -> VERB,VB
-PRON- -> PUNCT,HYPH
sustainable -> ADJ,JJ
asset -> NOUN,NN
over -> ADP,IN
the -> DET,DT
next -> ADJ,JJ
five -> NUM,CD
year -> NOUN,NN
with -> ADP,IN
climate -> NOUN,NN
change -> NOUN,NN
highlight -> NOUN,NN
as -> SCONJ,IN
the -> DET,DT
most -> ADV,RBS
prominent -> ADJ,JJ
sustainability -> NOUN,NN
issue -> NOUN,NN
. -> PUNCT,.


In [86]:
a,b,c = pos_extraction('sustainability target show result ten year with the AP fund Council on Ethics AP3 be well on -PRON- way to achieve the four sustaina - bility target set for 2018 where investment in green bond have triple since 2014 .')
pos_printer(a,b,c)

sustainability -> PROPN,NNP
target -> PROPN,NNP
show -> PROPN,NNP
result -> VERB,VBP
ten -> NUM,CD
year -> NOUN,NN
with -> ADP,IN
the -> DET,DT
AP -> PROPN,NNP
fund -> NOUN,NN
Council -> PROPN,NNP
on -> ADP,IN
Ethics -> PROPN,NNP
AP3 -> PROPN,NNP
be -> AUX,VB
well -> ADV,RB
on -> ADP,IN
-PRON- -> PUNCT,:
way -> NOUN,NN
to -> PART,TO
achieve -> VERB,VB
the -> DET,DT
four -> NUM,CD
sustaina -> NOUN,NN
- -> PUNCT,HYPH
bility -> NOUN,NN
target -> NOUN,NN
set -> VERB,VBN
for -> ADP,IN
2018 -> NUM,CD
where -> ADV,WRB
investment -> NOUN,NN
in -> ADP,IN
green -> ADJ,JJ
bond -> NOUN,NN
have -> AUX,VBP
triple -> ADJ,JJ
since -> SCONJ,IN
2014 -> NUM,CD
. -> PUNCT,.


In [159]:
a,b,c = pos_extraction('the carbon footprint of the company in -PRON- equity portfolio be at about the same level as in 2018 .')
pos_printer(a,b,c)

the -> DET,DT
carbon -> NOUN,NN
footprint -> NOUN,NN
of -> ADP,IN
the -> DET,DT
company -> NOUN,NN
in -> ADP,IN
-PRON- -> PUNCT,-LRB-
equity -> NOUN,NN
portfolio -> NOUN,NN
be -> AUX,VB
at -> ADP,IN
about -> ADP,IN
the -> DET,DT
same -> ADJ,JJ
level -> NOUN,NN
as -> SCONJ,IN
in -> ADP,IN
2018 -> NUM,CD
. -> PUNCT,.


In [156]:
lemmatization(['the carbon footprint of the companies in our equity portfolio was at about the same level as in 2018.'])

['the carbon footprint of the company in -PRON- equity portfolio be at about the same level as in 2018 .']

# Method

1. Store a list, keeping the number and extracted portion
> Alternative is to store a set and disregard the number in case multiple numbers in one sentence and we dedup the extracted sentences
2. Check left and right until we hit a verb not of finer pos tag 'VBP' or 'VB'. If in between this checking there are no NOUNS, continue checking after this VERB (exclude noun directly before and after). 
3. Get the indices of the tag we need to slice to retrieve from the tokens_list (must use this method since pos now has punctuations & stopwords)

In [145]:
special_ext_tokens = ['double', 'doubled', 'triple', 'tripled', 'half', 'quarter']

In [176]:
def extract_text(tags, verb_exclude): 
    tokens, pos_list, tag_list = tags[2], tags[3], tags[4]
    results = []
    for i in range(len(pos_list)):
        pos = pos_list[i]
        tag = tag_list[i]
        tok = tokens[i]
        if pos == 'NUM' and line_has_digits(tokens[i].text): #million recognised as a NUM
            j,k = extract_text_numbers(pos_list, tag_list, verb_exclude, i)
            results.append([tokens[i].text, generate_extracted_text(tokens, pos_list, j,k)])
        if (pos == 'DET' and tag == 'PDT') or tok.text in special_ext_tokens:
            j,k = extract_text_quant_words(pos_list, tag_list, i)
            results.append([tokens[i].text, generate_extracted_text(tokens, pos_list, j,k)])
    return results    

def extract_text_numbers(pos_list, tag_list, verb_exclude, i):
    j = max(i-1,0)
    k = min(i+1, len(pos_list)-1)    
    noun_flag_left, noun_flag_right = False, False
    if j != 0:
        while pos_list[j] != 'VERB' or noun_flag_left == False or tag_list[j] in verb_exclude:
            if pos_list[j] == 'NOUN':
                noun_flag_left = True
            j -= 1
            if j == 0:
                break
    if k != len(pos_list)-1:
        while pos_list[k] != 'VERB' or noun_flag_right == False or tag_list[k] in verb_exclude:
            if pos_list[k] == 'NOUN':
                noun_flag_right = True
            k += 1
            if k == len(pos_list)-1:
                break
    return j,k

def extract_text_quant_words(pos_list, tag_list, i):
    j = max(i-1,0)
    k = min(i+1, len(pos_list)-1)    
    adj_flag_left, adj_flag_right = False, False
    if j != 0:
        while pos_list[j] != 'NOUN' or adj_flag_left == False:
            if pos_list[j] == 'ADJ':
                adj_flag_left = True
            j -= 1
            if j == 0:
                break
    if k != len(pos_list)-1:
        while pos_list[k] != 'NOUN' or adj_flag_right== False:
            if pos_list[k] == 'ADJ':
                adj_flag_right = True
            k += 1
            if k == len(pos_list)-1:
                break
    return j,k
    
def generate_extracted_text(tokens, pos_list, j, k): #need to write if not simply joining will give extra spaces
    extracted_text = ''
    for tk in range(j,k):
        if pos_list[tk] != 'PUNCT' and pos_list[tk] != 'PART':
            extracted_text += ' '
        extracted_text += tokens[tk].text
    return extracted_text.strip()

def scoring_extraction(extracted_text, labelled_text, v_list):
    index = len(extracted_text)
    for i in range(index):
        v_list[0] += len(labelled_text[i].split(" "))
        v_list[1] += abs(len(extracted_text[i][1].split(" "))- len(labelled_text[i].split(" ")))
    return v_list

def print_accuracy(v):
    matched = round(100*(1-(v[1]/v[0])),2)
    print("The percentage of words accurately extracted is "+ str(matched)+ "%")

In [179]:
def extract_and_score_text(tags):
    l = []
    v0, v1, v2, v3 = [0,0], [0,0], [0,0], [0,0]
    for info in tags:
        result_noexclusion = extract_text(info, [])
        result_verbfg_exclusion1 = extract_text(info, ['VBP']) # best
        result_verbfg_exclusion2 = extract_text(info, ['VB'])
        result_verbfg_exclusion3 = extract_text(info, ['VBP', 'VB'])
        v0 = scoring_extraction(result_noexclusion, info[5], v0)
        v1 = scoring_extraction(result_verbfg_exclusion1, info[5], v1)
        v2 = scoring_extraction(result_verbfg_exclusion2, info[5], v2)
        v3 = scoring_extraction(result_verbfg_exclusion3, info[5], v3)
        l.append([info[0], result_noexclusion, result_verbfg_exclusion1, result_verbfg_exclusion2, result_verbfg_exclusion3])
    print_accuracy(v0)
    print_accuracy(v1)
    print_accuracy(v2)
    print_accuracy(v3)
    return l

In [180]:
print("print running lemma version")
df_lemma = extract_and_score_text(tags_lemma)
print("print running non-lemma version")
df_no_lemma = extract_and_score_text(tags_no_lemma)

print running lemma version
The percentage of words accurately extracted is 72.58%
The percentage of words accurately extracted is 77.06%
The percentage of words accurately extracted is 65.41%
The percentage of words accurately extracted is 67.11%
print running non-lemma version
The percentage of words accurately extracted is 72.85%
The percentage of words accurately extracted is 74.28%
The percentage of words accurately extracted is 73.03%
The percentage of words accurately extracted is 74.46%


In [181]:
pd.DataFrame(df_lemma).to_csv('test_final.csv')

-------

# Import Json

In [ ]:
f = open('../data/all_asian_banks_preprocessed_vfinal.json',) #can change to os directory method later to process all text
data = json.load(f) 

In [ ]:
data[17]

# Process Data

In [ ]:
# step 1 filtering, processing for relevant text
relevant_text = get_indices_filter_nondigits(data, decarb_terms)